In [3]:
#! pip install openai langchain faiss-cpu sentence-transformers
#! pip install -U langchain-community

In [1]:
# Step 1: Import Required Libraries for LLM + Document Retrieval Workflow
import os
import torch
from transformers import pipeline
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA

LangChain is a framework for building applications with language models (LLMs). 
- It simplifies the process of integrating LLMs with external data sources (like documents, databases, or APIs) 
- It allows you to create intelligent apps such as chatbots, retrieval-augmented generation (RAG) systems, and more.

LangChain acts as the "glue" that connects:

* Your documents (TextLoader)---------------------------------------> Loads plain text documents into LangChain.
* Your text processing (TextSplitter)-------------------------------> Splits long text into manageable chunks.
* Your embeddings (HuggingFaceEmbeddings)---------------------------> Generates embeddings for document chunks.
* Your retrieval system (FAISS)-------------------------------------> Stores and retrieves embeddings efficiently using similarity    search.It helps you find the most similar text chunks from your documents when someone asks a question.
* Your LLM (HuggingFacePipeline)------------------------------------> Wraps a Hugging Face model as an LLM.
* And your question-answering logic (RetrievalQA)-------------------> Connects retrieval and LLM to answer questions based on documents.

In [15]:
# Step 2: Load the document. A text loader is used to load the document as raw text into memory, so it can be processed further.

loader = TextLoader("llm_notes.txt", encoding="utf-8")
documents = loader.load()

# Step 3: Split the document. Each chunk has 300 characters, and 50 characters overlap with the next chunk for context continuity.

text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
docs = text_splitter.split_documents(documents[:5])  # limit size to reduce memory

# Step 4: Embed using Hugging Face sentence transformer

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
db = FAISS.from_documents(docs, embeddings)

# Step 5: Load flan-t5-small on CPU (safest config). Hugging Face’s pipeline is wrapped into a LangChain-compatible llm

device = torch.device("cpu")

qa_pipeline = pipeline(
    "text2text-generation",
    model="google/flan-t5-small",
    max_length=256,
    device=device,
    do_sample=False
)

llm = HuggingFacePipeline(pipeline=qa_pipeline)

# Step 6: Build the Retrieval QA chain. First retrieves top 3 relevant text chunks from FAISS, then passes them to the LLM to answer your query.
#It enhances the LLM's ability to answer questions by grounding it in specific documents.

retriever = db.as_retriever(search_kwargs={"k": 3})
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

# Step 7: Ask a question
#query = "What is tokenization in LLMs and why is it important?"
query = "What is the long-standing relationship between building codes and standards?"
result = qa_chain(query)

# Step 8: Show results

print("\n Answer:")
print(result["result"])

print("\n Source Documents:")
for i, doc in enumerate(result["source_documents"], 1):
    print(f"\n--- Source {i} ---")
    print(doc.page_content)


Device set to use cpu



 Answer:
A standard is a published technical document that represents a stakeholder consensus on how a material, product or assembly is to be designed, manufactured, tested or installed so that a construction

 Source Documents:

--- Source 1 ---
There is a long-standing relationship between building codes and standards that address design,
installation, testing and materials related to building construction. Building regulation cannot be
effectively carried out without such standards. The critical role of standards in the building

--- Source 2 ---
Standards and their relationship to the code
A standard is a published technical document that represents a stakeholder consensus on how a
material, product or assembly is to be designed, manufactured, tested or installed so that a

--- Source 3 ---
construction. When codes are adopted by units of government (i.e., towns, cities, counties,
states, or other agencies or jurisdictions with regulatory authority), they provide the legal
framewo

# Big Picture Flow

1- Load your notes.

2- Split into chunks.

3- Embed chunks into vectors.

4- Store vectors in FAISS (vector database).

5- Load an LLM (Flan-T5).

6- Build a chain: query → retrieve top chunks → pass to LLM → get grounded answer.

7- Print both the answer and the sources.

# Exercise

In [ ]:
! pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph
! pip install langchain
! pip install langchain-core
! pip install langchain-openai
! pip install langchain-community
! pip install langchain-experimental
! pip install langgraph
! pip install "langserve[all]"
! pip install langchain-cli
! pip install langsmith
! pip install -e .

In [4]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

KeyboardInterrupt: Interrupted by user

In [ ]:
! pip install -qU "langchain[openai]"

In [ ]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4o-mini", model_provider="openai")

In [ ]:
! pip install -qU langchain-openai

In [ ]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
! pip install -qU langchain-core

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [ ]:
import bs4
from langchain import hub
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# Load and chunk contents of the blog

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://example.com/page1",
    "https://example.com/page2",
]
loader = WebBaseLoader(
    web_paths= urls,
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,    # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
    )
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

# Index chunks
_ = vector_store.add_documents(documents=all_splits)

# Define prompt for question-answering
# N.B. for non-US LangSmith endpoints, you may need to specify
# api_url="https://api.smith.langchain.com" in hub.pull.
prompt = hub.pull("rlm/rag-prompt")


# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()